In [1]:
import os

In [2]:
import numpy as np
import pandas as pd
from sklearn import manifold


In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
if not os.path.exists('../data/lsaTestData.tar.gz'):
    !wget https://github.com/brian-cleary/LatentStrainAnalysis/raw/master/testData.tar.gz -O ../data/lsaTestData.tar.gz

In [5]:
!mkdir ../sandbox/lsaTestData_fnv_clf

In [6]:
os.chdir("../sandbox/lsaTestData_fnv_clf/")

In [7]:
!tar xf ../../data/lsaTestData.tar.gz >  /dev/null  2>&1

In [8]:
!cat original_reads/SRR*.fastq > data.fastq

In [23]:
!python ../../fastqToSeq.py -i data.fastq -o data.seq -s 1

In [24]:
!python ../../hashSeq.py -i data.seq --hash fnv -o data.fnv -k 15

parameters: {'out_file': 'data.fnv', 'hash_fun': 'fnv', 'in_file': 'data.seq', 'kmer_size': 15, 'n_thread': 7, 'hash_size': 22}
100%|████████████████████████████████| 200374/200374 [00:18<00:00, 10742.70it/s]


In [25]:
labels = pd.read_csv("data.seq",header=None, usecols=[1], sep='\t').iloc[:,0].map(lambda u: int(u.startswith("Spike"))).values

In [26]:
labels.mean()

0.10167985866429777

In [27]:
with open('data.fnv.clf','wt') as fout:
    with open('data.fnv') as fin:
        for i,line in enumerate(fin):
            label="__label__"+str(labels[i])
            fout.write(label+" "+line)

In [28]:
!wc -l data.fnv.clf

200374 data.fnv.clf


In [32]:
200374-160300

40074

In [31]:
!head -n 160300 data.fnv.clf > data.train

In [33]:
!tail -n 40074 data.fnv.clf > data.test

In [35]:
!../../fastseq supervised -input data.train -output model_data

Read 13M words
Number of words:  6537412
Number of labels: 2
Progress: 100.0% words/sec/thread:  960115 lr:  0.000000 loss:  0.018061 ETA:   0h 0m


In [36]:
! ../../fastseq test model_data.bin data.test          

N	40074
P@1	1
R@1	1
Number of examples: 40074


In [39]:
!../../fastseq supervised -input data.train -output model_data  -lr 1.0 -epoch 25 -wordNgrams 2 -bucket 200000 -dim 50 -loss hs

Read 13M words
Number of words:  6537412
Number of labels: 2
Progress: 100.0% words/sec/thread:  822155 lr:  0.000000 loss:  0.001958 ETA:   0h 0m


In [40]:
! ../../fastseq test model_data.bin data.test          

N	40074
P@1	1
R@1	1
Number of examples: 40074
